# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.shape

(551, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_df.reset_index()
ncities_df=cities_df.loc[cities_df["Max Temp"]<27]
ncities_df=ncities_df.loc[cities_df["Max Temp"]>21]
ncities_df=ncities_df.loc[cities_df["Wind Speed"]<10]
ncities_df=ncities_df.loc[cities_df["Cloudiness"]==0]
nan_value = float("NaN")
ncities_df.replace("", nan_value, inplace=True)
ncities_df.dropna(subset = ["Cloudiness"], inplace=True)
ncities_df.reset_index()
ncities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,6,saint-philippe,0.0,RE,1.603602e+09,61.0,-21.36,55.77,26.00,9.80
82,88,chabahar,0.0,IR,1.603602e+09,26.0,25.29,60.64,22.00,5.10
128,140,canakkale,0.0,TR,1.603602e+09,87.0,40.08,26.83,21.11,1.00
182,201,aswan,0.0,EG,1.603602e+09,35.0,24.09,32.91,22.26,6.96
262,296,benghazi,0.0,LY,1.603602e+09,63.0,32.12,20.07,21.05,3.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=[]
hotel_df=pd.DataFrame({"City":ncities_df["City"],
                      "Cloudiness":ncities_df["Cloudiness"],
                       "Country":ncities_df["Country"],
                        "Date":ncities_df["Date"],
                        "Humidity":ncities_df["Humidity"],
                        "Lat":ncities_df["Lat"],
                        "Lng":ncities_df["Lng"],
                        "Max Temp":ncities_df["Max Temp"],
                        "Wind Speed":ncities_df["Wind Speed"],
                        "Hotel Name":""
                      })
hotel_df.reset_index(inplace=True)

In [8]:
def gethotels(lat,lng):
    params = {
        "types": "lodging",
        "radius": 5000,
        "keyword": "hotel",
        #"rankby": "distance",
        "key": g_key,
        "location": f"{lat},{lng}"
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
        
    return response

In [10]:
results = []

for i, row in hotel_df.iterrows():
    function_response = gethotels(row["Lat"], row["Lng"])
    results.append(function_response)


In [11]:
for i,row in hotel_df.iterrows():
    try:
        hotel_df.loc[i,"Hotel Name"] = results[i]["results"][1]["name"]
    except:
        print(f"Missing result for {row['City']}")



Missing result for canakkale


In [12]:
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,6,saint-philippe,0.0,RE,1.603602e+09,61.0,-21.36,55.77,26.00,9.80,Pinpin d'Amour
1,82,chabahar,0.0,IR,1.603602e+09,26.0,25.29,60.64,22.00,5.10,Laleh Hotel
2,128,canakkale,0.0,TR,1.603602e+09,87.0,40.08,26.83,21.11,1.00,
3,182,aswan,0.0,EG,1.603602e+09,35.0,24.09,32.91,22.26,6.96,Hotel Sofitel Legend Old Cataract Aswan
4,262,benghazi,0.0,LY,1.603602e+09,63.0,32.12,20.07,21.05,3.72,فندق ترحاب
5,279,arlit,0.0,NE,1.603602e+09,19.0,18.74,7.39,22.53,4.10,Hôtel Temet
6,348,mocuba,0.0,MZ,1.603602e+09,65.0,-16.84,36.99,25.30,1.14,Residencial Vênus
7,361,rustenburg,0.0,ZA,1.603602e+09,49.0,-25.67,27.24,23.67,1.17,Safari Lodge Hotel & Convention Centre
8,382,fuerte olimpo,0.0,PY,1.603602e+09,75.0,-21.04,-57.87,25.18,1.68,Hotel Fuerte Olimpo
9,473,aquidauana,0.0,BR,1.603602e+09,79.0,-20.47,-55.79,24.70,1.46,Portal Pantaneiro Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
hotel_detail = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#markers = gmaps.marker_layer(locations2)
marker_layer = gmaps.marker_layer(locations2, info_box_content=hotel_detail)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))